In [1]:
import numpy as np
import pandas as pd

In [4]:
deliveries = pd.read_excel('Ball-by-Ball_T20International_Final.xlsx', index = False)

In [6]:
deliveries

,Match_Id,Year,Inning,Over,Batting Team,Batsman,Non-Striker,Bowler,Batsman Runs,Extra Runs,Total Runs,Wicket Type,Player Dismissed
0,263,2017,1,0.1,Australia,AJ Finch,M Klinger,SL Malinga,0,0,0,NaN,NaN
1,263,2017,1,0.2,Australia,AJ Finch,M Klinger,SL Malinga,0,0,0,NaN,NaN
2,263,2017,1,0.3,Australia,AJ Finch,M Klinger,SL Malinga,1,0,1,NaN,NaN
3,263,2017,1,0.4,Australia,M Klinger,AJ Finch,SL Malinga,2,0,2,NaN,NaN
4,263,2017,1,0.5,Australia,M Klinger,AJ Finch,SL Malinga,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120025,262,2016,2,17.1,Australia,TM Head,PM Nevill,SS Pathirana,1,0,1,NaN,NaN
120026,262,2016,2,17.2,Australia,PM Nevill,TM Head,SS Pathirana,3,0,3,NaN,NaN
120027,262,2016,2,17.3,Australia,TM Head,PM Nevill,SS Pathirana,0,0,0,NaN,NaN
120028,262,2016,2,17.4,Australia,TM Head,PM Nevill,SS Pathirana,0,0,0,NaN,NaN


In [16]:
def convertover(balls):
    num = int(balls/6)
    num = float(str(num) + '.' + str(balls % 6))
    return num

In [19]:
def bowler_statistics(Year, Start, Final):
    #Bowler-wise data for #balls, extra-runs, and Total runs conceeded
    deliveries_year = deliveries[deliveries['Match_Id'] <= Final]
    deliveries_year = deliveries_year[deliveries_year['Match_Id'] >= Start]
    deliveriesyear_cleaned = deliveries_year.drop(['Year', 'Batting Team','Non-Striker', 'Batsman', 'Wicket Type'], axis = 1)

    
    playerwise = deliveriesyear_cleaned.drop(['Match_Id', 'Inning', 'Batsman Runs', 'Player Dismissed'], axis = 1)
    playerwise.reset_index()
    playerwise['Over'] = 1
    
    playerwise = playerwise.groupby('Bowler').sum()
    
    #Grouped by Bowler - to find total number of wickets    
    wickets = deliveries_year.drop(['Batting Team', 'Non-Striker'], axis = 1)
    wickets.drop(['Batsman Runs', 'Extra Runs', 'Total Runs', 'Inning', 'Match_Id'], axis = 1, inplace=True)
    final_wickets = wickets.dropna(subset = ['Player Dismissed'])
    final_wickets['Player Dismissed'] = 1
    final_wickets.drop(['Over', 'Batsman'], axis = 1, inplace = True)
    final_wickets = final_wickets.groupby('Bowler').sum()
    
    wickettype = wickets.drop(['Over', 'Batsman', 'Player Dismissed'], axis = 1)
    wickettype.dropna(subset = ['Wicket Type'], inplace = True)
    wickettype['caught'] = np.zeros(len(wickettype))
    wickettype['bowled'] = np.zeros(len(wickettype))
    wickettype['caughtbowled']=np.zeros(len(wickettype))
    wickettype['lbw']=np.zeros(len(wickettype))
    wickettype['runout']=np.zeros(len(wickettype))
    wickettype['stumped']=np.zeros(len(wickettype))
    wickettype['others'] = np.zeros(len(wickettype))
    wickettype.reset_index(inplace=True)
    wickettype.drop('index',axis = 1, inplace = True)
    for i in range(0,len(wickettype)):
        if wickettype['Wicket Type'][i].lower() == 'caught':
            wickettype['caught'][i] = 1
        elif wickettype['Wicket Type'][i].lower() == 'bowled':
            wickettype['bowled'][i] = 1
        elif wickettype['Wicket Type'][i].lower() == 'caught and bowled':
            wickettype['caughtbowled'][i] = 1
        elif wickettype['Wicket Type'][i].lower() == 'lbw':
            wickettype['lbw'][i] = 1
        elif wickettype['Wicket Type'][i].lower() == 'run out':
            wickettype['runout'][i] = 1
        elif wickettype['Wicket Type'][i].lower() == 'stumped':
            wickettype['stumped'][i] = 1
        else:
            wickettype['others'][i] = 1
    

    wickettype.drop('Wicket Type', axis = 1, inplace = True)
    final_wickettype = wickettype.groupby('Bowler').sum()
    final_wickettype['No_Wickets'] = final_wickets['Player Dismissed']
    final_wickettype.reset_index(inplace = True)
    
    
    playerwise['caught'] = np.zeros(len(playerwise))
    playerwise['bowled'] = np.zeros(len(playerwise))
    playerwise['caughtbowled'] = np.zeros(len(playerwise))
    playerwise['lbw'] = np.zeros(len(playerwise))
    playerwise['runout'] = np.zeros(len(playerwise))
    playerwise['stumped'] = np.zeros(len(playerwise))
    playerwise['others'] = np.zeros(len(playerwise))
    playerwise['No_Wickets'] = np.zeros(len(playerwise))
    
    playerwise.reset_index(inplace = True)
    
    for i in range(0, len(playerwise)):
        for j in range(0, len(final_wickettype)):
            if playerwise['Bowler'][i] == final_wickettype['Bowler'][j]:
                playerwise['caught'][i] = final_wickettype['caught'][j]
                playerwise['bowled'][i] = final_wickettype['bowled'][j]
                playerwise['caughtbowled'][i] = final_wickettype['caughtbowled'][j]
                playerwise['lbw'][i] = final_wickettype['lbw'][j]
                playerwise['runout'][i] = final_wickettype['runout'][j]
                playerwise['stumped'][i] = final_wickettype['stumped'][j]
                playerwise['others'][i] = final_wickettype['others'][j]
                playerwise['No_Wickets'][i] = final_wickettype['No_Wickets'][j]
    playerwise.columns = ['Bowler', 'Balls Delivered', 'Extra Runs', 'Total Runs', 'caught', 'bowled',
       'caughtbowled', 'lbw', 'runout', 'stumped', 'others', 'No_Wickets']
    
    t20i_overs = playerwise
    
    #Bowling Average Calculation for Death Overs - Total Runs Conceeded / Wickets Taken
    t20i_overs['Bowling Average in Death Overs'] = 0
    for i in range(0,len(t20i_overs)):
        if t20i_overs['No_Wickets'][i] == 0:
            t20i_overs['Bowling Average in Death Overs'][i] = "-"
        else:
            t20i_overs['Bowling Average in Death Overs'][i] = round(t20i_overs['Total Runs'][i] / t20i_overs['No_Wickets'][i],1)
    
    #Bowling Strike Rate Calculation for Death Overs - Balls Delivered / Wickets Taken
    t20i_overs['Bowling SR in Death Overs'] = 0
    for i in range(0,len(t20i_overs)):
        if t20i_overs['No_Wickets'][i] == 0:
            t20i_overs['Bowling SR in Death Overs'][i] = "-"
        else:
            t20i_overs['Bowling SR in Death Overs'][i] = round(t20i_overs['Balls Delivered'][i] / t20i_overs['No_Wickets'][i],1)
    
    
    arr = t20i_overs['Balls Delivered'].apply(convertover)
    t20i_overs['Overs Bowled'] = arr
    
    #Bowling Economy Rate in Death Overs - Total Runs Conceeded / Overs bowled
    t20i_overs['Bowling Econ. Rate in Death Overs'] = round(t20i_overs['Total Runs'] / t20i_overs['Overs Bowled'],1)
    
    return t20i_overs

In [20]:
bowler_statistics(2013, 0, 50)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipy

,Bowler,Balls Delivered,Extra Runs,Total Runs,caught,bowled,caughtbowled,lbw,runout,stumped,others,No_Wickets,Bowling Average in Death Overs,Bowling SR in Death Overs,Overs Bowled,Bowling Econ. Rate in Death Overs
0,A Dananjaya,6,0,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-,-,1.0,12.0
1,AD Mathews,133,5,186,2.0,1.0,0.0,1.0,1.0,0.0,0.0,5.0,37.2,26.6,22.1,8.4
2,AD Russell,25,1,36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-,-,4.1,8.8
3,AJ Finch,6,0,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-,-,1.0,9.0
4,AK Perera,6,0,17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-,-,1.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,WB Rankin,25,6,29,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,29,25,4.1,7.1
183,WD Parnell,140,11,162,6.0,1.0,0.0,1.0,0.0,0.0,0.0,8.0,20.2,17.5,23.2,7.0
184,XJ Doherty,67,3,77,2.0,1.0,0.0,0.0,0.0,1.0,0.0,4.0,19.2,16.8,11.1,6.9
185,Ziaur Rahman,13,2,26,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,26,13,2.1,12.4


In [21]:
bowler_statistics(2014, 51, 111)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7

,Bowler,Balls Delivered,Extra Runs,Total Runs,caught,bowled,caughtbowled,lbw,runout,stumped,others,No_Wickets,Bowling Average in Death Overs,Bowling SR in Death Overs,Overs Bowled,Bowling Econ. Rate in Death Overs
0,A Bhattarai,19,2,13,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,6,9,3.1,4.2
1,A Mishra,135,9,153,5.0,1.0,0.0,1.0,1.0,3.0,0.0,11.0,13,12,22.3,6.9
2,AD Mathews,135,2,140,4.0,1.0,0.0,1.0,0.0,0.0,0.0,6.0,23,22,22.3,6.3
3,AD Russell,155,9,219,5.0,4.0,0.0,0.0,1.0,0.0,0.0,10.0,21,15,25.5,8.6
4,AF Milne,72,2,57,3.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,14,18,12.0,4.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,WD Parnell,144,12,186,9.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,20.7,16,24.0,7.8
176,Wahab Riaz,12,0,15,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,15,12,2.0,7.5
177,Yuvraj Singh,6,0,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-,-,1.0,13.0
178,Ziaur Rahman,28,1,64,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,64,28,4.4,14.5
